In [34]:
from bs4 import BeautifulSoup
import requests
import re   #clean headers

In [8]:
#connect to link
url = 'https://en.wikipedia.org/wiki/List_of_African_countries_by_population'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [21]:
tables = soup.find_all('table', class_= 'wikitable sortable')


In [35]:
# Iterate through each table and find table titles
for table in tables:
    # Find all headers in the table
    headers = table.find_all('th')
    # Extract text from each header, clean it, and create a list of headers
    header_texts = [re.sub(r'\[\d+\]', '', header.get_text(strip=True)) for header in headers]

In [36]:
#store the titles in a pandas dataframe
import pandas as pd
df = pd.DataFrame(columns=header_texts)
df

,,Country,%Africa,Africapopulation,%growth,Officialfigure,Officialdate,


In [68]:
rows = table.find_all('tr')  
# Iterate through each row and extract the data
for row in rows[1:]:  # Skip the header row
    row_data = row.find_all('td')
    cells = [re.sub(r'\[.*?\]', '', data.get_text(strip=True)) for data in row_data]
     # Check if the number of cells matches the number of headers
    if len(cells) == len(header_texts):
        df.loc[len(df)] = cells
    else:
        # Handle cases where the row does not match the header length
        print(f"Skipping row due to length mismatch: {cells}")

# Print a sample of the DataFrame to check the result
df.sample(15)

Skipping row due to length mismatch: ['', 'Total', '100%', '1,458,384,742', '2.4%', '1,332,291,870', '']


,,Country,%Africa,Africapopulation,%growth,Officialfigure,Officialdate,
297,22,Zambia,1.4%,"20,569,738",2.8%,"19,610,769",14 Sep 2022,
481,41,Gambia,0.2%,"2,773,168",2.5%,"2,417,471",1 Jul 2022,
157,48,Mauritius,0.1%,"1,300,557",0.1%,"1,261,196",31 Dec 2022,
194,30,Burundi,0.9%,"13,238,559",2.7%,"13,097,400",1 Jul 2023,
521,26,Zimbabwe,1.1%,"16,665,409",2.1%,"15,178,979",20 Apr 2022,
582,32,South Sudan,0.8%,"11,088,796",1.6%,"13,249,924",1 Jul 2020,
121,12,Angola,2.5%,"36,684,203",3.1%,"34,094,077",1 Jul 2023,
299,24,Somalia,1.2%,"18,611,700",3.1%,"12,316,895",4 May2024[28],[b]
114,5,Tanzania,4.6%,"69,419,073",3.0%,"61,741,120",23 Aug 2022,
100,46,Guinea-Bissau,0.1%,"2,150,842",2.2%,"1,781,308",1 Jul 2023,


In [71]:
# Define a function to clean and extract the date from a string
def extract_date(text):
    # Regular expression to match dates in the format "1 Jul 2020"
    date_match = re.search(r'\d{1,2} \w{3} \d{4}', text)
    return date_match.group(0) if date_match else text  
# Clean the 'Officialdate' column if it exists
if 'Officialdate' in df.columns:
    df['Officialdate'] = df['Officialdate'].apply(extract_date)

In [72]:
df.sample(15)

,,Country,%Africa,Africapopulation,%growth,Officialfigure,Officialdate,
63,9,Sudan,3.3%,"48,109,006",2.6%,"41,984,500",1 Jul 2018,
28,29,Benin,0.9%,"13,712,828",2.7%,"12,606,998",1 Jul 2023,
419,34,Sierra Leone,0.6%,"8,791,092",2.2%,"7,541,641",9 Dec 2021,
511,16,Ivory Coast,2.0%,"28,873,034",2.5%,"29,389,150",14 Dec 2021,
499,4,DR Congo,7.0%,"102,262,809",3.3%,"95,370,000",1 Jul 2019,
377,47,Equatorial Guinea,0.1%,"1,714,672",2.4%,"1,558,160",1 Jul 2022,
476,36,Republic of the Congo,0.4%,"6,222,913",2.3%,"3,697,490",4 May2024[42],
66,12,Angola,2.5%,"36,684,203",3.1%,"34,094,077",1 Jul 2023,
47,48,Mauritius,0.1%,"1,300,557",0.1%,"1,261,196",31 Dec 2022,
176,12,Angola,2.5%,"36,684,203",3.1%,"34,094,077",1 Jul 2023,


In [79]:
df.to_csv('WorldPopulation.csv', index = False)